# Names and types translation

There are numerous names and types translation options in the [options](https://github.com/pthom/litgen/blob/main/src/litgen/options.py) file.

Below is a relevant extract from the options:

```python
    ################################################################################
    #    <names translation from C++ to python>
    ################################################################################
    # Convert variables, functions and namespaces names to snake_case (class, structs, and enums names are always preserved)
    python_convert_to_snake_case: bool = True
    # List of code replacements when going from C++ to Python
    # Notes:
    # - by default, type_replacements is prefilled with standard_type_replacements()
    #   type_replacements will be applied to all types (including class and enum names)
    # - by default, value_replacements is prefilled with standard_value_replacements()
    # - by default, comments_replacements is prefilled with standard_comments_replacements()
    # - by default, the others are empty
    # - type_replacements, var_names_replacements and function_names_replacements enable you
    #   to modify the outputted python code
    type_replacements: RegexReplacementList  # = cpp_to_python.standard_type_replacements() by default
    var_names_replacements: RegexReplacementList  # = RegexReplacementList() by default (i.e. empty)
    namespace_names_replacements: RegexReplacementList  # = RegexReplacementList() by default (i.e. empty)
    function_names_replacements: RegexReplacementList  # = RegexReplacementList() by default (i.e. empty)
    value_replacements: RegexReplacementList  # = cpp_to_python.standard_value_replacements() by default
    comments_replacements: RegexReplacementList  # = cpp_to_python.standard_comment_replacements() by default
    macro_name_replacements: RegexReplacementList  # = RegexReplacementList() by default (i.e. empty)
```

## Types replacements

`options.type_replacements` enables to change the way some types are exported.

Let's take an example with some C++ code. 

In the example below, `MyPair` is a template class that should behave like a std::pair, and should be presented as a python `Tuple[int, int]`. 

In [1]:
cpp_code = """
    MyPair<int, int> GetMinMax(std::vector<int>& values);
"""

If we convert it, we see that `std::vector<int>` is correctly interpreted as a `List[int]`, however `MyPair<int, int>` is not.

In [2]:
import litgen
from litgen.demo import litgen_demo

options = litgen.LitgenOptions()
litgen_demo.demo(options, cpp_code)

In order to account for `MyPair<int, int>`, we need to add replacements to `options.type_replacements`:

In [3]:
options = litgen.LitgenOptions()
options.type_replacements.add_last_replacement(
    r"MyPair<(.*),\s*(.*)>", # this is a regex, with 2 captures
    r"Tuple[\1, \2]"         # and this is the replacement
)
litgen_demo.demo(options, cpp_code)

Note: by default `options.type_replacements` already contains standard replacements regexes. See `standard_type_replacements()` inside packages/litgen/internal/cpp_to_python.py.

## Convert to snake case

The code below:

In [4]:
cpp_code = """
namespace MyNamespace
{
    struct MyClass
    {
        int AddNumber(int a, int b);

        int MultiplierRatio = 4;
    };
}
"""

By default, the bindings will convert CamelCase to snake_case for functions, variables and namespaces:

In [5]:
import litgen

options = litgen.LitgenOptions()
litgen_demo.demo(options, cpp_code)

However, you can set `options.python_convert_to_snake_case = False` to disable this.

In [6]:
options.python_convert_to_snake_case = False
litgen_demo.demo(options, cpp_code)